In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import dateparser

np.random.seed(1620)

In [ ]:
dataset_path = Path("~/Desktop/data/train.tsv")
metadata_cols = ["claimID", "claim", "label", "claimURL",
           "reason", "categories", "speaker", "checker", 
           "tags", "articleTitle", "publishDate", "claimDate", "entities"]

In [ ]:
df_rev = pd.read_csv(dataset_path, sep="\t", names=metadata_cols, 
                     na_values=["None","['None']"], parse_dates=['publishDate','claimDate'])


In [ ]:
df_rev['publishDate'].value_counts()

In [ ]:
df_rev['publishDate'].dropna().apply(dateparser.parse)

In [ ]:
dateparser.parse('Sun 8 May 2016, 7:37am')

In [ ]:
df_rev.isna().sum()

Than marks interesting columns and delete rows that contains nans

In [ ]:
essential_cols = ["articleTitle","checker","speaker","claimDate"] # columns subset that must not contain any null in any case
df_rev = df_rev.dropna(subset=essential_cols) # clean NAN values
len(df_rev)

In [ ]:
df_rev.isna().sum()

Now we can fill nan value for "reason" column with random sampled values

In [ ]:
justifications = ["Missing Context","Deceptive Editing", "Malicious Transformation"]
new_reasons=np.random.choice(justifications,size=df_rev['reason'].isna().sum()) # generate random sample of justifications

In [ ]:
df_rev.assign(reason=new_reasons)
df_rev = df_rev.reset_index()
df_rev